# Säätietojen poiminta, lataus ja muuntaminen (ELT)

## 1. Säätietojen poiminta (extract)

Poimitaan säätiedot Ilmatieteenlaitoksen API:sta. Tiedot tulevat melko suurina XML-dokumentteina, jotka tallennetaan tiedostoihin muistin säästämiseksi.

In [2]:
#install.packages("XML")
library(XML)

In [3]:
get_times_and_destination_filenames <- function() {
    return (data.frame(
                starttime = c("2021-01-01T02:00:00Z", "2021-02-01T02:00:00Z", "2021-03-01T02:00:00Z"
                              , "2021-04-01T02:00:00Z", "2021-05-01T02:00:00Z", "2021-06-01T02:00:00Z"),
                endtime = c("2021-02-01T01:00:00Z", "2021-03-01T01:00:00Z", "2021-04-01T01:00:00Z"
                            , "2021-05-01T01:00:00Z", "2021-06-01T01:00:00Z", "2021-07-01T01:00:00Z"),
                file = c("2021_1.xml", "2021_2.xml", "2021_3.xml", "2021_4.xml", "2021_5.xml", "2021_6.xml")
            ))
}

get_filename <- function() {
    
}

create_source_url <- function(place, starttime, endtime) {
    return (paste(
                "http://opendata.fmi.fi/wfs/fin?service=WFS&version=2.0.0&request=GetFeature"
                , "&storedquery_id=fmi::observations::weather::hourly::simple&place="
                , place
                , "&starttime="
                , starttime
                , "&endtime="
                , endtime
                , "&", sep="")
           )
}

get_file_with_path <- function(filename) {
    return paste( ".\\loaded_data\\", filename, sep="")
}

save_xml_from_url <- function(url, destination_file) {
    xml.content <- xmlTreeParse(url)
    saveXML(xmlRoot(xml.content), file=destination_file)
}

Data täytyy poimia osissa Ilmatieteenlaitoksen API:n rajoitusten takia

In [ ]:
xml.url.place = "Helsinki"

times_and_files <- get_times_and_destination_filenames()

In [23]:
extract.execute <- function(times_and_files, xml.url.place) {
    extract.starttime <- Sys.time()
    
    for (i in 1:nrow(times_and_files)) {
        xml.url = create_source_url(place = xml.url.place
                                    , starttime = as.character(times_and_files[["starttime"]][i])
                                    , endtime = as.character(times_and_files[["endtime"]][i])
                                   )
        save_xml_from_url(url = xml.url
                          , destination_file = get_file_with_path(as.character(times_and_files[["file"]][i]))
                         )
    }
    
    extract.endtime <- Sys.time() 
    print(extract.endtime - extract.starttime)
}

Proseduurin ajo on kommentoitu, jotta Ilmatieteenlaitos APIin kohdistuisi mahdollisimman vähän turhia kyselyitä.
Poistetaan kommentointi, kun halutaan ladata tiedot.

In [ ]:
#extract.execute(times_and_files, xml.url.place)

## 2.Säätietojen lataaminen (load)
> TODO: tallennetaan xml-dokumentit tietokantaan levyn sijaan, jolloin ne on helpommin saatavilla myöhempää käyttöä varten

---

## 3. Tietojen muuntaminen (transform)
Muunnetaan tiedot helpommin käsiteltävään tietomalliin. Tässä tarkoituksena on selväkielistää säätiedoissa olevat koodit ja muodostaa taulu, jonka jyvänä on kaikki yhden tunnin sisällä tehdyt mittaukset.

In [15]:
#install.packages("odbc")
library(odbc)
library(dplyr)

Windows koneella lisättynä tietokantayhteys SQL Serveriin "ODBC Data Sources"-sovelluksesta. Tietokanta on valmiiksi luotuna.

In [16]:
data_conn <- DBI::dbConnect(odbc::odbc(), "DATASOURCE", database = "SaaDB")

### 3.1 XML-dokumentin sisältö tietokantatauluun

Luodaan tietokannan taulut

In [17]:
create_table_saatiedot_raaka <- function() {
    fields_raaka <- c('INT IDENTITY(1,1) NOT NULL', 'VARCHAR(50) NOT NULL', 'DATETIME NOT NULL', 'VARCHAR(20) NOT NULL', 'VARCHAR(20)')
    names(fields_raaka) <- c('ID', 'PAIKKA', 'AIKA', 'PARAMETRIN_NIMI', 'PARAMETRIN_ARVO')
    
    DBI::dbCreateTable(data_conn, "SAATIEDOT_RAAKA", fields_raaka)
    DBI::dbExecute(data_conn, 'ALTER TABLE SAATIEDOT_RAAKA ADD CONSTRAINT [SAATIEDOT_RAAKA_PK] PRIMARY KEY ([ID])')
    }

#Poista tarvittaessa taulu
DBI::dbExecute(data_conn, 'DROP TABLE SAATIEDOT_RAAKA')
create_table_saatiedot_raaka()

[1] 0

[1] 0

Haetaan tiedot XML-tiedostoista ja tallennetaan ne tauluun.

In [18]:
saatiedot_raaka <- DBI::dbReadTable(data_conn, "SAATIEDOT_RAAKA")
# Id generoidaan taulussa automaattisesti [IDENTITY(1,1)], joten sitä ei täytetä täällä. Poistetaan se virheiden välttämiseksi.
saatiedot_raaka <- subset(saatiedot_raaka, select=-c(ID))

In [19]:
#XML-dokumentissa käytetään nimiavaruuksia, joita tarvitaan dataa lukiessa.
namespaces <- c(wfs='http://www.opengis.net/wfs/2.0', BsWfs='http://xml.fmi.fi/schema/wfs/2.0', gml='http://www.opengis.net/gml/3.2')

In [20]:
xml.content = xmlParse(file = paste( ".\\loaded_data\\", as.character(times_and_files[["file"]][1]), sep=""))
rootnode = xmlRoot(xml.content)
print(rootnode[[8000]][[1]])

<BsWfs:BsWfsElement id="BsWfsElement.1.667.8">
  <BsWfs:Location>
    <gml:Point id="BsWfsElementP.1.667.8" srsDimension="2" srsName="http://www.opengis.net/def/crs/EPSG/0/4258">
      <gml:pos>60.17523 24.94459</gml:pos>
    </gml:Point>
  </BsWfs:Location>
  <BsWfs:Time>2021-01-28T20:00:00Z</BsWfs:Time>
  <BsWfs:ParameterName>WD_PT1H_AVG</BsWfs:ParameterName>
  <BsWfs:ParameterValue>50.0</BsWfs:ParameterValue>
</BsWfs:BsWfsElement> 


In [21]:
format_time <- function(time) {
    modified_time <- sub('T', ' ', time)
    modified_time
    } 

In [22]:
load_data_to_dbtable <- function() {
    load_starttime=Sys.time()

    for (i in 1:nrow(times_and_files)) {
        xml.content = xmlParse(file = paste( ".\\loaded_data\\", as.character(times_and_files[["file"]][i]), sep=""))
        rootnode = xmlRoot(xml.content)
        for (j in 1:xmlSize(rootnode)) {
            result.time <- format_time(xpathApply(rootnode[[j]][[1]],path='BsWfs:Time',xmlValue,namespaces=namespaces))
            result.location <- xpathApply(rootnode[[j]][[1]],path='BsWfs:Location//gml:Point//gml:pos',xmlValue,namespaces=namespaces)
            result.parametername <- xpathApply(rootnode[[j]][[1]],path='BsWfs:ParameterName',xmlValue,namespaces=namespaces)
            result.parametervalue <- xpathApply(rootnode[[j]][[1]],path='BsWfs:ParameterValue',xmlValue,namespaces=namespaces)

            saatiedot_raaka[nrow(saatiedot_raaka)+1,] <- c(paikka=result.location, aika=result.time, parametrin_nimi=result.parametername, parametrin_arvo=result.parametervalue)

            if(nrow(saatiedot_raaka) >= 1000) {
                DBI::dbAppendTable(data_conn, "SAATIEDOT_RAAKA",saatiedot_raaka)
                saatiedot_raaka <- subset(saatiedot_raaka, 1==0) 
            }
        }
    }
    DBI::dbAppendTable(data_conn, "SAATIEDOT_RAAKA",saatiedot_raaka)

    print(Sys.time() - load_starttime)
    }

load_data_to_dbtable()

Time difference of 2.069225 mins


### 3.2 Tietomalli
Muunnetaan suoraan XML-dokumentista haetut tiedot tietomalliin ja selväkielistetään koodit.